In [2]:

import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm

sns.set_palette('colorblind')

DATADIR = os.path.expanduser("~/data/usgs_water")

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [3]:
df = pd.read_csv(os.path.join(DATADIR, "usgs_water_short.csv"))

/tmp/ipykernel_466580/1209995151.py:1: DtypeWarning: Columns (0,1,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(DATADIR, "usgs_water_short.csv"))


In [7]:
df["yr_unit_abbrv_tx"].value_counts()

gal/yr           236289
Mgal/d           184770
acre-feet/yr      90972
Mgal/yr           87743
Tgal/yr            8512
acre-feet/m        5905
Tacre-feet/yr      2039
Tgal/d              335
gal/m                89
Tcf/yr                5
Name: yr_unit_abbrv_tx, dtype: int64

In [18]:
def convert_to_m3_per_year(value, unit):
    # Conversion factors to m3/year
    conversion_factors = {
        'gal/yr': 0.00378541,  # 1 gallon = 0.00378541 m3
        'Mgal/d': 1.3815016e6,  # 1 million gallons per day = 1.3815016e6 m3/year
        'acre-feet/yr': 1233.48183754752,  # 1 acre-foot per year = 1233.48183754752 m3/year
        'Mgal/yr': 3785.41,  # 1 million gallons per year = 3785.41 m3/year
        'Tgal/yr': 3.78541,  # 1 thousand gallons per year = 3.78541 m3/year
        'acre-feet/m': 1233.48183754752 * 12,  # 1 acre-foot per month = 1233.48183754752 * 12 m3/year
        'Tacre-feet/yr': 1.23348183754752e6,  # 1 thousand acre-feet per year 
        'Tgal/d': 1.3815016e3,  # 1 thousand gallons per day = 1.3815016 m3/year
        'gal/m': 0.00378541 * 12,  # 1 gallon per month = 0.00378541 * 12 m3/year
        'Tcf/yr': 2.8316846592e2  # 1 thousand cubic feet per year 
    }

    if unit not in conversion_factors:
        raise ValueError(f"Unknown unit: {unit}")
    
    return value * conversion_factors[unit]


In [19]:
df["cn_qnty_m3_yr"] = df.apply(lambda row: convert_to_m3_per_year(row["cn_qnty_yr_va"], row["yr_unit_abbrv_tx"]), axis=1)

In [20]:
df["cn_qnty_m3_yr"].describe()

count    6.166590e+05
mean     2.576990e+08
std      4.427976e+09
min     -3.740968e+07
25%      9.325136e+03
50%      7.483134e+04
75%      4.006355e+05
max      2.987427e+12
Name: cn_qnty_m3_yr, dtype: float64

In [22]:
df.sort_values("cn_qnty_m3_yr").tail(n=12)

,from_site_owner_alias_nm,from_site_owner_cd,from_naics_ds,from_nat_aqfr_cd,from_county_nm,from_state_nm,to_site_owner_alias_nm,cn_qnty_yr_va,yr_unit_abbrv_tx,mo_unit_abbrv_tx,cn_qnty_m3_yr
167036,NaN,NaN,NaN,NaN,Hughes County,South Dakota,NaN,1.536050e+04,Mgal/d,Mgal/d,2.122056e+10
167037,NaN,NaN,NaN,NaN,Hughes County,South Dakota,NaN,1.536050e+04,Mgal/d,Mgal/d,2.122056e+10
167040,NaN,NaN,NaN,NaN,Charles Mix County,South Dakota,NaN,1.717634e+04,Mgal/d,Mgal/d,2.372914e+10
167041,NaN,NaN,NaN,NaN,Charles Mix County,South Dakota,NaN,1.717634e+04,Mgal/d,Mgal/d,2.372914e+10
167043,NaN,NaN,NaN,NaN,Yankton County,South Dakota,NaN,1.935284e+04,Mgal/d,Mgal/d,2.673598e+10
167042,NaN,NaN,NaN,NaN,Yankton County,South Dakota,NaN,1.935284e+04,Mgal/d,Mgal/d,2.673598e+10
597706,CRISP COUNTY POWER COMM,GV,Hydroelectric Power Generation,NaN,Worth County,Georgia,CRISP COUNTY POWER COMM,2.234500e+04,Mgal/d,Mgal/d,3.086965e+10
154389,NaN,NaN,Water Supply and Irrigation Systems,NaN,San Diego County,California,NaN,1.230000e+00,Tcf/yr,Tcf/yr,3.482972e+10
155163,NaN,NaN,Water Supply and Irrigation Systems,NaN,Tuolumne County,California,NaN,2.301000e+01,Tcf/yr,Tcf/yr,6.515706e+11
153700,NaN,NaN,Water Supply and Irrigation Systems,NaN,Alameda County,California,NaN,2.406251e+01,Tcf/yr,Tcf/yr,6.813743e+11


In [25]:
np.quantile(df["cn_qnty_m3_yr"], [0, 0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99, 1])

array([-3.74096818e+07,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        9.32513580e+03,  7.48313371e+04,  4.00635464e+05,  2.62349622e+06,
        1.48979337e+08,  5.26817564e+09,  2.98742732e+12])